In [210]:
import pandas as pd
from utilities import readJSON
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

In [152]:
plt.rcParams["figure.figsize"] = (20,10)

In [153]:
atomGroups = '/media/stephen/Data/modelling/GNG12-v2/GNG12/2000/reps/1/lammps/config/atomGroups.json'
atomGroups = readJSON(atomGroups)
TUbeads = atomGroups['TU']

In [154]:
TADstatus = '/media/stephen/Data/modelling/GNG12-v2/GNG12/2000/reps/1/beadTADstatus.csv.gz'
TADstatus = pd.read_csv(TADstatus)

In [220]:
TUinfo = '/media/stephen/Data/modelling/GNG12-v2/GNG12/2000/reps/1/TU-info.csv.gz'
TUinfo = pd.read_csv(TUinfo)

In [221]:
TUinfo

,timestep,ID,type,x,y,z,TFnear,active,TADstatus,TADgroup,time
0,0,26,5,36.29208,-35.66294,-15.27331,13.897148,False,0,0,0.0
1,0,27,5,36.67324,-35.37377,-14.48157,14.582054,False,0,0,0.0
2,0,482,10,-0.22754,-1.35844,11.39898,17.294629,False,0,0,0.0
3,0,483,5,0.09548,-0.54607,10.92229,17.933538,False,0,0,0.0
4,0,486,10,1.27824,-0.24361,8.44512,17.222039,False,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1813,100000,1160,5,22.15539,17.54981,29.35542,11.214147,False,0,4,1000.0
1814,100000,1161,10,22.90258,18.13789,29.71219,10.361831,False,0,4,1000.0
1815,100000,1564,10,45.31141,9.76422,22.02341,1.242169,True,0,12,1000.0
1816,100000,1565,10,45.20431,9.69246,22.98612,1.668129,True,0,12,1000.0


In [159]:
TUbeadStatus = TADstatus[TADstatus['ID'].isin(TUbeads)]

In [214]:
TUstats = pd.merge(TUinfo, TUbeadStatus)

In [204]:
df = allTUinfo.pivot(index='timestep', columns='ID', values='active')
df = df.corr('pearson').stack()
df.index = df.index.set_names(['ID1', 'ID2'])
df = df.reset_index()

In [207]:
pd.merge

,timestep,ID,type,x,y,z,TFnear,active,TADstatus,TADgroup,time
0,0,26,5,36.29208,-35.66294,-15.27331,13.897148,False,0,0,0.0
1,0,27,5,36.67324,-35.37377,-14.48157,14.582054,False,0,0,0.0
2,0,482,10,-0.22754,-1.35844,11.39898,17.294629,False,0,0,0.0
3,0,483,5,0.09548,-0.54607,10.92229,17.933538,False,0,0,0.0
4,0,486,10,1.27824,-0.24361,8.44512,17.222039,False,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1813,100000,1160,5,22.15539,17.54981,29.35542,11.214147,False,0,4,1000.0
1814,100000,1161,10,22.90258,18.13789,29.71219,10.361831,False,0,4,1000.0
1815,100000,1564,10,45.31141,9.76422,22.02341,1.242169,True,0,12,1000.0
1816,100000,1565,10,45.20431,9.69246,22.98612,1.668129,True,0,12,1000.0


In [263]:
# Generate matrix of all TU pairs across timepoints
allTUpairs = (TUstats.groupby('timestep')['ID']
  .apply(lambda x: pd.DataFrame((i for i in combinations(x.values,2))))\
  .reset_index().drop('level_1',axis=1)\
  .rename(columns={0: 'TU1', 1: 'TU2'}))

In [254]:
allTUpairs = pd.merge(
    allTUpairs, TUstats[['ID', 'timestep', 'TADstatus', 'TADgroup', 'active']],
    left_on=['timestep', 'TU1'], right_on=['timestep', 'ID']).drop(['ID'], axis=1)
allTUpairs.rename(
    columns={'TADstatus': 'TADstatus-TU1', 'TADgroup': 'TADgroup-TU1', 'active': 'active-TU1'},
    inplace=True)
allTUpairs = pd.merge(
    allTUpairs, TUstats[['ID', 'timestep', 'TADstatus', 'TADgroup', 'active']],
    left_on=['timestep', 'TU2'], right_on=['timestep', 'ID']).drop(['ID'], axis=1)
allTUpairs.rename(
    columns={'TADstatus': 'TADstatus-TU2', 'TADgroup': 'TADgroup-TU2', 'active': 'active-TU2'},
    inplace=True)

In [261]:
allTUpairs['TADstatus'] = allTUpairs.apply(setPairStatus, axis=1)

In [244]:
def setPairStatus(row):
    """ Label TU-TU pair status by relative TAD positions """
    seperation = abs(abs(row['TADgroup-TU1']) - abs(row['TADgroup-TU2']))
    TU1_inTAD = row['TADstatus-TU1'] > 0
    TU2_inTAD = row['TADstatus-TU2'] > 0
    if seperation == 0:
        if TU1_inTAD:
            return 'sameTAD'
        else:
            return 'sameNotTAD'
    elif seperation == 1:
        if TU1_inTAD == TU2_inTAD:
            if TU1_inTAD:
                return 'adjacentTAD'
            else:
                return 'adjacentNotTAD'
        else:
            return 'adjacentBoundary'
    else:
        return 'other'


In [251]:
allTUpairs = pd.merge(
    allTUpairs, TUstats[['ID', 'timestep', 'TADstatus', 'TADgroup', 'active']],
    left_on=['timestep', 'TU1'], right_on=['timestep', 'ID']).drop(['ID'], axis=1)

In [262]:
allTUpairs.groupby('TADstatus')

,timestep,TU1,TU2,TADstatus-TU1,TADgroup-TU1,active-TU1,TADstatus-TU2,TADgroup-TU2,active-TU2,TADstatus
0,0,26,27,0,0,False,0,0,False,sameNotTAD
1,0,26,482,0,0,False,0,0,False,sameNotTAD
2,0,27,482,0,0,False,0,0,False,sameNotTAD
3,0,26,483,0,0,False,0,0,False,sameNotTAD
4,0,27,483,0,0,False,0,0,False,sameNotTAD
...,...,...,...,...,...,...,...,...,...,...
15448,100000,1159,1683,0,4,False,0,14,False,other
15449,100000,1160,1683,0,4,False,0,14,False,other
15450,100000,1161,1683,0,4,False,0,14,False,other
15451,100000,1564,1683,0,12,True,0,14,False,other


In [264]:
allTUpairs

,timestep,TU1,TU2
0,0,26,27
1,0,26,482
2,0,26,483
3,0,26,486
4,0,26,701
...,...,...,...
15448,100000,1161,1565
15449,100000,1161,1683
15450,100000,1564,1565
15451,100000,1564,1683
